In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *
from glob import glob
from sklearn.model_selection import train_test_split
from model import build_model
from utils import *
from metrics import *


In [2]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [3]:

def read_image(x):
    x = x.decode()
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    return image

def read_mask(y):
    y = y.decode()
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def parse_data(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = np.concatenate([y, y], axis=-1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([224, 224, 3])
    y.set_shape([224, 224, 2])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=32)
    dataset = dataset.map(map_func=parse_data)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch)
    return dataset



# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)
    create_dir("files")

    train_path = "data_comb1/train/"
    valid_path = "data_comb1/validation/"

    ## Training
    train_x = sorted(glob(os.path.join(train_path, "img", "*.png")))
    train_y = sorted(glob(os.path.join(train_path, "masks_machine", "*.png")))

    ## Shuffling
    train_x, train_y = shuffling(train_x, train_y)

    ## Validation
    valid_x = sorted(glob(os.path.join(valid_path, "img", "*.png")))
    valid_y = sorted(glob(os.path.join(valid_path, "masks_machine", "*.png")))
    #model.h5 is the original weight file; new.ht is trained with (700 images?)
    model_path = "files/CVC-612model-org.h5"
    batch_size = 3
    epochs = 20
    lr = 1e-4
    shape = (224, 224, 3)

    model = build_model(shape)
    metrics = [
        dice_coef,
        iou,
        Recall(),
        Precision()
    ]
    
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
    
    #model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)
    #model.compile(loss=binary_crossentropy, optimizer=Nadam(lr), metrics=metrics
    model.compile(loss=bce_dice_loss, optimizer=Adam(lr), metrics=metrics)
    model.summary()



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

In [4]:
    callbacks = [
        #ModelCheckpoint(model_path),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20),
        CSVLogger("files/data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
        cp_callback
    ]

    train_steps = (len(train_x)//batch_size)
    valid_steps = (len(valid_x)//batch_size)

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

In [5]:

    model.fit(train_dataset,
            epochs=epochs,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            callbacks=callbacks,
            shuffle=False)
    model.save("5videos_20_da5.h5")

Epoch 1/20
  1/933 [..............................] - ETA: 0s - loss: 1.7175 - dice_coef: 0.0626 - iou: 0.0323 - recall: 0.5200 - precision: 0.0303WARNING:tensorflow:From d:\anaconda\envs\tf2-segmentation3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
933/933 [==============================] - ETA: 0s - loss: 1.1846 - dice_coef: 0.1513 - iou: 0.0827 - recall: 0.7448 - precision: 0.3804
Epoch 00001: saving model to cp.ckpt
933/933 [==============================] - 590s 632ms/step - loss: 1.1846 - dice_coef: 0.1513 - iou: 0.0827 - recall: 0.7448 - precision: 0.3804 - val_loss: 0.9726 - val_dice_coef: 0.2139 - val_iou: 0.1210 - val_recall: 0.7899 - val_precision: 0.6080
Epoch 2/20
933/933 [==============================] - ETA: 0s - loss: 0.7766 - dice_coef: 0.3452 - iou: 0.2138 - recall: 0.7976 - precis

Epoch 19/20
933/933 [==============================] - ETA: 0s - loss: 0.1343 - dice_coef: 0.8897 - iou: 0.8029 - recall: 0.8897 - precision: 0.9073
Epoch 00019: saving model to cp.ckpt
933/933 [==============================] - 355s 381ms/step - loss: 0.1343 - dice_coef: 0.8897 - iou: 0.8029 - recall: 0.8897 - precision: 0.9073 - val_loss: 0.1751 - val_dice_coef: 0.8579 - val_iou: 0.7562 - val_recall: 0.8651 - val_precision: 0.8744
Epoch 20/20
933/933 [==============================] - ETA: 0s - loss: 0.1244 - dice_coef: 0.8978 - iou: 0.8159 - recall: 0.8987 - precision: 0.9129
Epoch 00020: saving model to cp.ckpt
933/933 [==============================] - 356s 381ms/step - loss: 0.1244 - dice_coef: 0.8978 - iou: 0.8159 - recall: 0.8987 - precision: 0.9129 - val_loss: 0.1806 - val_dice_coef: 0.8534 - val_iou: 0.7498 - val_recall: 0.8587 - val_precision: 0.8771


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)


In [6]:
model.fit(train_dataset,
        epochs=2,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle=False)
model.save("5videos_20_da3.h5")

Epoch 1/2
  1/933 [..............................] - ETA: 0s - loss: 0.0731 - dice_coef: 0.9269 - iou: 0.8638 - recall: 0.9188 - precision: 0.9357WARNING:tensorflow:From d:\anaconda\envs\tf2-segmentation3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
933/933 [==============================] - ETA: 0s - loss: 0.1183 - dice_coef: 0.8817 - iou: 0.7912 - recall: 0.8721 - precision: 0.8990
Epoch 00001: saving model to cp.ckpt
933/933 [==============================] - 371s 397ms/step - loss: 0.1183 - dice_coef: 0.8817 - iou: 0.7912 - recall: 0.8721 - precision: 0.8990 - val_loss: 0.1423 - val_dice_coef: 0.8577 - val_iou: 0.7560 - val_recall: 0.8598 - val_precision: 0.8685
Epoch 2/2
933/933 [==============================] - ETA: 0s - loss: 0.1121 - dice_coef: 0.8879 - iou: 0.8007 - recall: 0.8796 - precisio

In [6]:
print("Number of layers in the base model: ", len(model.layers))


Number of layers in the base model:  221
